# UKB Bloodcells Multivariate fine-mapping

## Analysis

### Extract Data

```
m=/project2/mstephens/yuxin/ukb-bloodcells/zscores
cd $m && ls *.rds | sed 's/\.rds//g' > analysis_units.txt && cd -
sos run /project2/mstephens/yuxin/mvarbvs/analysis/multivariate/20201221_ukb_ED_prior.ipynb extract_effects \
    --cwd /project2/mstephens/yuxin/ukb-bloodcells  --analysis-units $m/analysis_units.txt \
    --datadir $m &> extract_effects.log
sos run /project2/mstephens/yuxin/mvarbvs/analysis/multivariate/20201221_ukb_ED_prior.ipynb Ycor \
    --cwd /project2/mstephens/yuxin/ukb-bloodcells
sos run /project2/mstephens/yuxin/mvarbvs/analysis/multivariate/20201221_ukb_ED_prior.ipynb Ycov \
    --cwd /project2/mstephens/yuxin/ukb-bloodcells
sos run /project2/mstephens/yuxin/mvarbvs/analysis/multivariate/20201221_ukb_ED_prior.ipynb zcor \
    --cwd /project2/mstephens/yuxin/ukb-bloodcells
```

### Compute priors

```
sos run analysis/20210503_ukb_pipeline.ipynb factor_analysis
sos run analysis/20210503_ukb_pipeline.ipynb mixture_model
```

### Prepare sufficient and summary statistics

```
sos run analysis/20210503_ukb_analysis.ipynb ldstore_to_rds \
    --ld_dir /project2/mstephens/yuxin/ukb-bloodcells/LD --name bloodcells_chr \
    --wd /project2/mstephens/yuxin/ukb-bloodcells/LD_ldstore \
    -c ~/GitHub/mvarbvs/midway2.yml -q midway2

sos run analysis/20210503_ukb_analysis.ipynb sufficient_summary_stats_preprocessing \
    --analysis_units /project2/mstephens/yuxin/ukb-bloodcells/region_units.txt \
    --data_dir /project2/mstephens/yuxin/ukb-bloodcells/genotypes \
    --data_suffix pgen \
    --phenoFile /project2/mstephens/yuxin/ukb-bloodcells/bloodcells.pheno.txt \
    --covarFile /project2/mstephens/yuxin/ukb-bloodcells/bloodcells.covar.txt \
    --z_dir /project2/mstephens/yuxin/ukb-bloodcells/zscores --z_suffix z.rds \
    --ld_dir /project2/mstephens/yuxin/ukb-bloodcells/LD --ld_suffix ld.rds \
    --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
    -c ~/GitHub/mvarbvs/midway2.yml -q bigmem2
```

### Fine-mapping

```
sos run analysis/20210503_ukb_pipeline.ipynb univariate_analysis
sos run analysis/20210503_ukb_pipeline.ipynb suff_analysis
sos run analysis/20210503_ukb_pipeline.ipynb rss_analysis
```

Get data name that need to rerun with larger L
```
sos run ~/GitHub/mvarbvs/analysis/20210503_ukb_analysis.ipynb CS_report \
    --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
    --data_suffix Ycov.mvsusiesuff.rds \
    --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
    --analysis_units /project2/mstephens/yuxin/ukb-bloodcells/region_units.txt
cd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result
cat **/*.Ycov.mvsusiesuff.summary.md | sort -g -k2 | awk '$2 == 10 {print $1}' > ../regions_Ycov.mvsusiesuff.L10.txt

sos run ~/GitHub/mvarbvs/analysis/20210503_ukb_analysis.ipynb CS_report \
    --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
    --data_suffix LDoriginal.zcor.mvsusierss.rds \
    --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
    --analysis_units /project2/mstephens/yuxin/ukb-bloodcells/region_units.txt
cd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result
cat **/*.LDoriginal.zcor.mvsusierss.summary.md | sort -g -k2 | awk '$2 == 10 {print $1}' > ../regions_LDoriginal.zcor.mvsusierss.L10.txt

sos run ~/GitHub/mvarbvs/analysis/20210503_ukb_analysis.ipynb CS_report \
    --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
    --data_suffix LDoriginal.Ycor.mvsusierss.rds \
    --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
    --analysis_units /project2/mstephens/yuxin/ukb-bloodcells/region_units.txt
cd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result
cat **/*.LDoriginal.Ycor.mvsusierss.summary.md | sort -g -k2 | awk '$2 == 10 {print $1}' > ../regions_LDoriginal.Ycor.mvsusierss.L10.txt

sos run ~/GitHub/mvarbvs/analysis/20210503_ukb_analysis.ipynb CS_report \
    --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
    --data_suffix LDremove_cov.zcor.mvsusierss.rds \
    --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
    --analysis_units /project2/mstephens/yuxin/ukb-bloodcells/region_units.txt
cd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result
cat **/*.LDremove_cov.zcor.mvsusierss.summary.md | sort -g -k2 | awk '$2 == 10 {print $1}' > ../regions_LDremove_cov.zcor.mvsusierss.L10.txt

sos run ~/GitHub/mvarbvs/analysis/20210503_ukb_analysis.ipynb CS_report \
    --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
    --data_suffix LDremove_cov.Ycor.mvsusierss.rds \
    --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
    --analysis_units /project2/mstephens/yuxin/ukb-bloodcells/region_units.txt
cd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result
cat **/*.LDremove_cov.Ycor.mvsusierss.summary.md | sort -g -k2 | awk '$2 == 10 {print $1}' > ../regions_LDremove_cov.Ycor.mvsusierss.L10.txt

```

## Summary
```
sos run analysis/20210503_ukb_pipeline.ipynb summary    
```

In [ ]:
[global]
parameter: analysis_units = path('/project2/mstephens/yuxin/ukb-bloodcells/region_units.txt')
parameter: wd = path('/project2/mstephens/yuxin/ukb-bloodcells/')
parameter: name = "ukbbloodcells_prepare"
parameter: max_L = 10

In [ ]:
[factor_analysis]
script: interpreter= 'qsub', expand = True
    #!/bin/bash

    #SBATCH --time=36:00:00
    #SBATCH --partition=mstephens
    #SBATCH --nodes=1
    #SBATCH --ntasks-per-node=1
    #SBATCH --cpus-per-task=1
    #SBATCH --mem-per-cpu=2000
    #SBATCH --job-name={step_name}
    #SBATCH --mail-type=BEGIN,END,FAIL

    module load R
    sos run ~/GitHub/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb flash \
        --name {name} --cwd {wd} -s build &> {wd}/{name}_factor_analysis.log
    sos run ~/GitHub/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb pca \
        --name {name} --cwd {wd} -s build &>> {wd}/{name}_factor_analysis.log
    sos run ~/GitHub/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb canonical \
        --name {name} --cwd {wd} -s build &>> {wd}/{name}_factor_analysis.log

In [ ]:
[mixture_model]
def get_cmd(m):
    c1 = f'''
    sos run ~/GitHub/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb ud \
        --ud-method ed --resid_cor /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_prepare.Ycor.rds \
        --mixture_components flash pca canonical \
        --name {m} --cwd {wd} \
        -c ~/GitHub/mvarbvs/midway2.yml -q stephenslab -s build &> {wd}/ed_Y_{m}.log
    '''
    c2 = f'''
    sos run ~/GitHub/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb ud \
        --ud-method ed \
        --mixture_components flash pca canonical \
        --name {m} --cwd {wd} \
        -c ~/GitHub/mvarbvs/midway2.yml -q stephenslab -s build &> {wd}/ed_znull_{m}.log
    '''
    
    c3 = f'''
    sos run ~/GitHub/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb ud \
        --ud-method teem --resid_cor /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_prepare.Ycor.rds \
        --mixture_components flash pca canonical \
        --name {m} --cwd {wd} \
        -c ~/GitHub/mvarbvs/midway2.yml -q stephenslab -s build &> {wd}/teem_Y_{m}.log
    '''
    c4 = f'''
    sos run ~/GitHub/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb ud \
        --ud-method teem \
        --mixture_components flash pca canonical \
        --name {m} --cwd {wd} \
        -c ~/GitHub/mvarbvs/midway2.yml -q stephenslab -s build &> {wd}/teem_znull_{m}.log
    '''
    
    c5 = f'''
    sos run ~/GitHub/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb ed \
          --resid_cor /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_prepare.Ycor.rds \
          --mixture_components flash pca canonical \
          --name {m} --cwd {wd} \
        -c ~/GitHub/mvarbvs/midway2.yml -q stephenslab -s build &> {wd}/bovy_Y_{m}.log
    '''
    c6 = f'''
    sos run ~/GitHub/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb ed \
          --mixture_components flash pca canonical \
          --name {m} --cwd {wd} \
          -c ~/GitHub/mvarbvs/midway2.yml -q stephenslab -s build &> {wd}/bovy_znull_{m}.log
    '''
    return [c1,c2,c3,c4,c5,c6]
cmds = get_cmd(name)
input: for_each = 'cmds'
script: interpreter= 'qsub', expand = True
#!/bin/bash
  
#SBATCH --time=36:00:00
#SBATCH --partition=mstephens
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=2000
#SBATCH --job-name={step_name}
#SBATCH --mail-type=BEGIN,END,FAIL

module load R
{_cmds}

In [1]:
[univariate_analysis]
def get_cmd():
    c1 = f'''
    sos run /project2/mstephens/yuxin/mvarbvs/analysis/20210503_ukb_analysis.ipynb univariate_analysis \
        --analysis_units {analysis_units} \
        --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
        --data_suffix sufficient_stats.rds \
        --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result_20210508 \
        --max_L {max_L} \
        -c ~/GitHub/mvarbvs/midway2.yml -q bigmem2 -s build &> {wd}/univariate_analysis.log
    '''
    return [c1]
cmds = get_cmd()
input: for_each = 'cmds'
script: interpreter= 'qsub', expand = True
#!/bin/bash
  
#SBATCH --time=36:00:00
#SBATCH --partition=mstephens
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=2000
#SBATCH --job-name={step_name}
#SBATCH --mail-type=BEGIN,END,FAIL

module load R
{_cmds}

SyntaxError: invalid syntax (<ipython-input-1-a51ff9b68bc4>, line 14)

In [ ]:
[suff_analysis]
def get_cmd():
    c1 = f'''
    sos run /project2/mstephens/yuxin/mvarbvs/analysis/20210503_ukb_analysis.ipynb sufficient_stats_analysis \
        --analysis_units {analysis_units} \
        --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
        --data_suffix sufficient_stats.rds \
        --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
        --prior /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_prepare.ed_bovy.rds \
        --resid_cor /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_prepare.Ycov.rds \
        --max_L {max_L} \
        -c ~/GitHub/mvarbvs/midway2.yml -q midway2 -s build &> {wd}/suff_analysis.log
    '''
    return [c1]
cmds = get_cmd()
input: for_each = 'cmds'
script: interpreter= 'qsub', expand = True
#!/bin/bash
  
#SBATCH --time=36:00:00
#SBATCH --partition=mstephens
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=2000
#SBATCH --job-name={step_name}
#SBATCH --mail-type=BEGIN,END,FAIL

module load R
{_cmds}

In [ ]:
[rss_analysis]
def get_cmd():
    c1 = f'''
    sos run /project2/mstephens/yuxin/mvarbvs/analysis/20210503_ukb_analysis.ipynb summary_stats_analysis \
        --analysis_units {analysis_units} \
        --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
        --data_suffix summary_stats.rds \
        --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
        --prior /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_prepare.ed_bovy.rds \
        --resid_cor /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_prepare.Ycor.rds \
        --ld_type original --max_L {max_L} \
        -c ~/GitHub/mvarbvs/midway2.yml -q midway2 -s build &> {wd}/rss_analysis_VYcor_ldoriginal.log
    '''
    
    c2 = f'''
    sos run /project2/mstephens/yuxin/mvarbvs/analysis/20210503_ukb_analysis.ipynb summary_stats_analysis \
        --analysis_units {analysis_units} \
        --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
        --data_suffix summary_stats.rds \
        --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
        --prior /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_prepare.ed_bovy.rds \
        --resid_cor /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_prepare.zcor.rds \
        --ld_type original --max_L {max_L} \
        -c ~/GitHub/mvarbvs/midway2.yml -q midway2 -s build &> {wd}/rss_analysis_Vzcor_ldoriginal.log
    '''
    
    c3 = f'''
    sos run /project2/mstephens/yuxin/mvarbvs/analysis/20210503_ukb_analysis.ipynb summary_stats_analysis \
        --analysis_units {analysis_units} \
        --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
        --data_suffix summary_stats.rds \
        --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
        --prior /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_prepare.ed_bovy.rds \
        --resid_cor /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_prepare.Ycor.rds \
        --ld_type remove_cov --max_L {max_L} \
        -c ~/GitHub/mvarbvs/midway2.yml -q midway2 -s build &> {wd}/rss_analysis_VYcor_ldremove_cov.log
    '''
    
    c4 = f'''
    sos run /project2/mstephens/yuxin/mvarbvs/analysis/20210503_ukb_analysis.ipynb summary_stats_analysis \
        --analysis_units {analysis_units} \
        --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
        --data_suffix summary_stats.rds \
        --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
        --prior /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_prepare.ed_bovy.rds \
        --resid_cor /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_prepare.zcor.rds \
        --ld_type remove_cov --max_L {max_L} \
        -c ~/GitHub/mvarbvs/midway2.yml -q midway2 -s build &> {wd}/rss_analysis_Vzcor_ldremove_cov.log
    '''
    return [c1,c2,c3,c4]
cmds = get_cmd()
input: for_each = 'cmds'
script: interpreter= 'qsub', expand = True
#!/bin/bash
  
#SBATCH --time=36:00:00
#SBATCH --partition=mstephens
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=2000
#SBATCH --job-name={step_name}
#SBATCH --mail-type=BEGIN,END,FAIL

module load R
{_cmds}

In [ ]:
[summary]
def get_cmd():
    c1 = f'''
    sos run ~/GitHub/mvarbvs/analysis/20210503_ukb_analysis.ipynb analysis_report \
        --analysis_units /project2/mstephens/yuxin/ukb-bloodcells/region_units.txt \
        --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
        --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
        --data_suffix Ycov.mvsusiesuff.rds \
        -s build > mvsusiesuff_summary.log
    '''
    c2 = f'''
    sos run ~/GitHub/mvarbvs/analysis/20210503_ukb_analysis.ipynb analysis_report \
        --analysis_units /project2/mstephens/yuxin/ukb-bloodcells/region_units.txt \
        --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
        --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
        --data_suffix LDremove_cov.Ycor.mvsusierss.rds \
        -s build > mvsusierss_LDremove_cov_Ycor_summary.log
    '''
    c3 = f'''
    sos run ~/GitHub/mvarbvs/analysis/20210503_ukb_analysis.ipynb analysis_report \
        --analysis_units /project2/mstephens/yuxin/ukb-bloodcells/region_units.txt \
        --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
        --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
        --data_suffix LDremove_cov.zcor.mvsusierss.rds \
        -s build > mvsusierss_LDremove_cov_zcor_summary.log
    '''
    c4 = f'''
    sos run ~/GitHub/mvarbvs/analysis/20210503_ukb_analysis.ipynb analysis_report \
        --analysis_units /project2/mstephens/yuxin/ukb-bloodcells/region_units.txt \
        --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
        --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
        --data_suffix LDoriginal.Ycor.mvsusierss.rds \
        -s build > mvsusierss_LDoriginal_Ycor_summary.log
    '''
    c5 = f'''
    sos run ~/GitHub/mvarbvs/analysis/20210503_ukb_analysis.ipynb analysis_report \
        --analysis_units /project2/mstephens/yuxin/ukb-bloodcells/region_units.txt \
        --data_dir /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_stats \
        --wd /project2/mstephens/yuxin/ukb-bloodcells/ukbbloodcells_result \
        --data_suffix LDoriginal.zcor.mvsusierss.rds \
        -s build > mvsusierss_LDoriginal_zcor_summary.log
    '''
    return [c1,c2,c3,c4,c5]
cmds = get_cmd()
input: for_each = 'cmds'
script: interpreter= 'qsub', expand = True
#!/bin/bash
  
#SBATCH --time=36:00:00
#SBATCH --partition=mstephens
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=55G
#SBATCH --job-name={step_name}
#SBATCH --mail-type=BEGIN,END,FAIL

module load R
{_cmds}